[Claude chat](https://claude.ai/chat/d5cba3e2-bb37-48e8-918a-690f57373367)

In [1]:
# Cell 1: Install required packages
# %pip install -r ../requirements.txt 


In [2]:

# Cell 2: Imports and Setup
import os
from typing import List
import PyPDF2
from crewai import Agent, Task, Crew
from langchain.llms import Ollama
import warnings
warnings.filterwarnings('ignore')

# Initialize Ollama LLM
llm = Ollama(model="llama3.2")


ModuleNotFoundError: No module named 'PyPDF2'

In [ ]:

# Cell 3: PDF Content Extractor
def extract_pdf_content(pdf_path: str) -> str:
    """Extract text content from a PDF file."""
    content = ""
    try:
        with open(pdf_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            for page in pdf_reader.pages:
                text = page.extract_text()
                if text:
                    content += text
        return content
    except Exception as e:
        raise Exception(f"Error reading PDF: {str(e)}")


In [ ]:
# Cell 4: Create Agents
def create_agents(llm):
    """Create specialized agents for test generation."""
    question_creator = Agent(
        name="Question Creator",
        role="test question creator",  # Ensure this matches available role options
        goal="Create varied and effective test question from the analyzed content",
        backstory="Expert in creating diverse types of test questions, skilled at assessing different cognitive levels.",
        llm=llm,
        verbose=True
    )

    format_specialist = Agent(
        name="Format Specialist",
        role="test format specialist",  # Adjusted to match the expected role name
        goal="Create a well-structured and properly formatted test",
        backstory="Expert in organizing test content and ensuring proper formatting.",
        llm=llm,
        verbose=True
    )

    return question_creator, format_specialist


In [ ]:

# Cell 5: Create Tasks
def create_tasks(pdf_content: str, question_creator, format_specialist) -> List[Task]:
    """Create sequential tasks for test generation."""
    
    generate_questions_task = Task(
        description=f"""
        Analyze the following content and create 20 test questions:
        {pdf_content[:1000]}...  # Truncated for brevity
        Ensure questions:
        - Cover main topics and key concepts
        - Include different difficulty levels
        - Test different cognitive skills
        """,
        agent=question_creator,
        expected_output="List of test questions in JSON format."
    )

    format_test_task = Task(
        description="Organize questions by type, add instructions, and ensure consistent JSON formatting.",
        agent=format_specialist,
        dependencies=[generate_questions_task],
        expected_output="A well-formatted test ready for review."
    )

    return [generate_questions_task, format_test_task]


In [ ]:

# Cell 6: Main Test Generation Function
def generate_test(pdf_path: str) -> str:
    """Execute the test generation process."""
    try:
        # Extract PDF content
        pdf_content = extract_pdf_content(pdf_path)
        
              
        # Create agents
        question_creator, format_specialist = create_agents(llm)
        
        # Create tasks
        tasks = create_tasks(pdf_content, question_creator, format_specialist)
        
        # Create and run the crew
        crew = Crew(
            agents=[question_creator, format_specialist],
            tasks=tasks,
            verbose=True
        )
        
        # Run the process
        result = crew.kickoff()
        
        return result
    
    except Exception as e:
        raise Exception(f"Error generating test: {str(e)}")

In [ ]:

# Cell 7: Example Usage
# Replace with your PDF path
PDF_PATH = "data/Lowther.pdf"

# Generate test
try:
    test = generate_test(PDF_PATH)
    print("Test generated successfully!")
    print("\nGenerated Test:")
    print("-" * 50)
    print(test)
    
    # Optionally save to file
    with open("generated_test.txt", "w") as f:
        f.write(test)
    print("\nTest saved to 'generated_test.txt'")
    
except Exception as e:
    print(f"Error: {str(e)}")

 [DEBUG]: == Working Agent: test question creator
 [INFO]: == Starting Task: 
        Analyze the following content and create 20 test questions:
        AIR & SPACE OPERATIONS REVIEW   3
UNMANNED  AIRCRAFT  SYSTEMSCombat Drones in Ukraine
AdAm Lowther
mAhbube  K. S iddiKi
Drones are playing an important role in the war in Ukraine. Without a large conventional air 
force, the Ukrainian military is employing a number of high- and low- end imported and domes -
tically produced drones to devastating effect against Russian forces. This article examines how 
Ukrainian and Russian forces are employing these drones and their effects on the battlefield.
Ukrainian resistance to Russia’s invasion surprised not only Russian president 
Vladimir Putin but also Western intelligence agencies and prominent analysts.1 
A wide range of drones are among the celebrated systems proving effective for 
Ukrainian forces, most notably the Baykar Bayraktar TB2. This combat drone now has a 
song and music video 